In [1]:
from selenium import webdriver
import datetime, re, requests, io, time, random, string
from bs4 import BeautifulSoup
from credentials import username, password
from pymongo import MongoClient
client = MongoClient('localhost', 27017)

In [9]:
urls = {
    'sign_in': 'https://wallmine.com/users/sign-in',
    'homepage': 'https://wallmine.com/',
    'view_stock': lambda exchange, symbol: f'https://wallmine.com/{exchange}/{symbol}'
}
urls['view_stock']('nasdaq', 'tsla')

'https://wallmine.com/nasdaq/tsla'

In [10]:
urls['sign_in']

'https://wallmine.com/users/sign-in'

In [6]:
driver = webdriver.Chrome('/Users/amirharrison/downloads/chromedriver')

In [7]:
def login():
    driver.get(urls['sign_in'])
    time.sleep(3)
    driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[1]/div[2]/a').click()
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="user_email"]').send_keys(username)
    driver.find_element_by_xpath('//*[@id="user_password"]').send_keys(password)
    time.sleep(.2)
    driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[2]/div[1]/button').click()
    time.sleep(3)
    if driver.find_element_by_xpath('/html/body/main/section/div[2]/div/div[1]/h1/span'):print('Login Was Successful')

In [11]:
login()

Login Was Successful


In [12]:
def retrieve_data():
#     we need to be on the site after login in order for this method to work
    driver.find_element_by_xpath('//*[@id="homepage-heatmap"]/a/div[2]').click()
    time.sleep(5)
    text = driver.find_element_by_xpath('/html/body/main/section/div[4]/div/div/div[1]/div/div[1]/h1').text
    driver.find_element_by_xpath('/html/body/main/section/div[5]/form/div/div/div').click()
    if text == 'Free Stock Screener':
        print('On the right page')
    driver.find_element_by_xpath('/html/body/main/section/div[5]/div/div/div[1]/div/ul/li[1]/a').click()
    time.sleep(2)
    table_head = driver.find_element_by_xpath('/html/body/main/section/div[5]/div/div/div[2]/table/thead').text
    table_body = driver.find_element_by_xpath('/html/body/main/section/div[5]/div/div/div[2]/table/tbody').text
    raw_data = [table_head, table_body]
    
    return raw_data

In [13]:
got_data = retrieve_data()

On the right page


In [14]:
clean_data_head = got_data[0].split('\n')
clean_data_head

['▴ ▴',
 'Symbol',
 '▴ ▴',
 'Company',
 '▴ ▴',
 'Exchange',
 '▴ ▴',
 'Industry',
 '▴ ▴',
 'Market cap',
 '▴ ▴',
 'EBITDA',
 '▴ ▴',
 'P/E',
 '▴ ▴',
 'EV/EBITDA',
 '▴ ▴',
 'Debt/Equity',
 '▴ ▴',
 'Average volume',
 '▴ ▴',
 'Institutional ownership',
 '▴ ▴',
 'Earnings date',
 '▴ ▴',
 'Price',
 '▴ ▴',
 'Performance today']

In [15]:
def remove_characters(x):
    if x == '▴ ▴':
        return False
    else:
        return True

In [16]:
categories = list(filter(remove_characters, clean_data_head))
categories

['Symbol',
 'Company',
 'Exchange',
 'Industry',
 'Market cap',
 'EBITDA',
 'P/E',
 'EV/EBITDA',
 'Debt/Equity',
 'Average volume',
 'Institutional ownership',
 'Earnings date',
 'Price',
 'Performance today']

In [17]:
clean_data_body = got_data[1].split('\n')
clean_data_body[0]

'AAPL Apple Inc NASDAQ Consumer Electronics $1.97T $85.159B 32.21 24.07 3.96 114.562M 59.66% 27 Jan 2021 $121.21 +0.51%'

In [30]:
split_company = clean_data_body[0].split('$')
split_company

['AAPL Apple Inc NASDAQ Consumer Electronics ',
 '1.97T ',
 '85.159B 32.21 24.07 3.96 114.562M 59.66% 27 Jan 2021 ',
 '121.21 +0.51%']

In [20]:
split_symbol = split_company[0].split(' ')
split_symbol

['AAPL', 'Apple', 'Inc', 'NASDAQ', 'Consumer', 'Electronics', '']

In [21]:
symbol = (f'{categories[0]}: {split_symbol[0]}')
symbol

'Symbol: AAPL'

In [22]:
company_split =  split_company[0].split('NASDAQ')
company_split

['AAPL Apple Inc ', ' Consumer Electronics ']

In [23]:
price_split = split_company[3].split(' ')
price_split

['121.21', '+0.51%']

In [24]:
price = (f'{categories[12]}: ${price_split[0]}')
price

'Price: $121.21'

In [26]:
def if_nas_or_nyse(x):
    if x == 'NASDAQ':
        return True
    elif x == 'NYSE':
        return True
    elif x == 'x':
        return True
    else:
        return False

In [55]:
split_company = clean_data_body[0].split('$')
split_company

['AAPL Apple Inc NASDAQ Consumer Electronics ',
 '1.97T ',
 '85.159B 32.21 24.07 3.96 114.562M 59.66% 27 Jan 2021 ',
 '121.21 +0.51%']

In [56]:
def parse_company(split_company):
    result = {}
    split_row_one = split_company[0].split(' ')
    split_row_two = split_company[1].split(' ')
    split_row_three = split_company[2].split(' ')
    split_row_four = split_company[3].split(' ')
    stop = ['NASDAQ', 'NYSE', 'NYSEMKT']
    symbol = split_row_one[0]
    price = split_row_four[0]
    for n in stop:
        if n in split_company[0]:
            exchange = n
    a = split_row_one.index(exchange)
    company_name_arr = split_row_one[1:a]
    company_name = ' '.join(str(e) for e in company_name_arr)
    result['company_name'] = company_name
    result['exchange'] = exchange
    result['symbol'] = symbol
    result['price'] = price
    result['date'] = datetime.datetime.now()
    return result # object/dictionary

In [57]:
parse_company(split_company)

{'company_name': 'Apple Inc',
 'exchange': 'NASDAQ',
 'symbol': 'AAPL',
 'price': '121.21',
 'date': datetime.datetime(2021, 3, 26, 17, 51, 10, 65706)}

In [64]:
all_company_data = []
for stock in clean_data_body:
    if 'N/A' in stock:
        print('this works')
        continue
    elif '¥' in stock:
        continue
    elif '€' in stock:
        continue
    split_company = stock.split('$')
    
    x = parse_company(split_company)
    all_company_data.append(x)

this works
this works
this works
this works
this works
this works
this works
this works
this works
this works
this works
this works
this works
this works
this works
this works
this works
this works
this works
this works


In [66]:
final_list = [categories, all_company_data]
final_list

[['Symbol',
  'Company',
  'Exchange',
  'Industry',
  'Market cap',
  'EBITDA',
  'P/E',
  'EV/EBITDA',
  'Debt/Equity',
  'Average volume',
  'Institutional ownership',
  'Earnings date',
  'Price',
  'Performance today'],
 [{'company_name': 'Apple Inc',
   'exchange': 'NASDAQ',
   'symbol': 'AAPL',
   'price': '121.21',
   'date': datetime.datetime(2021, 3, 26, 17, 53, 26, 368599)},
  {'company_name': 'Microsoft Corporation',
   'exchange': 'NASDAQ',
   'symbol': 'MSFT',
   'price': '236.48',
   'date': datetime.datetime(2021, 3, 26, 17, 53, 26, 368615)},
  {'company_name': 'Amazon.com Inc.',
   'exchange': 'NASDAQ',
   'symbol': 'AMZN',
   'price': '3,052.03',
   'date': datetime.datetime(2021, 3, 26, 17, 53, 26, 368626)},
  {'company_name': 'Facebook Inc',
   'exchange': 'NASDAQ',
   'symbol': 'FB',
   'price': '282.96',
   'date': datetime.datetime(2021, 3, 26, 17, 53, 26, 368920)},
  {'company_name': 'Berkshire Hathaway Inc.',
   'exchange': 'NYSE',
   'symbol': 'BRK-B',
   'pri